In [10]:
from __future__ import division
import os,time
import tensorflow as tf
import tensorflow.keras.layers as l
import numpy as np


<h1>DenseNets Tiramisu.</h1>
<h3>Отныне и впредь я воссоздаю эту архитектуру модели. Выбор пал на неё из-за её превосходства над другими моделями сегментации. Данная модель способна отлично сегментировать разные данные. Модель будет перенесена и адаптирована мной для улучшения выполнения задачи. Спасибо за внимание!</h3>
<h6>Код для переработки был взят из <a href='https://github.com/ShashiAI/FC-DenseNet-Tiramisu'>этого источника</a></h6>

In [ ]:
def preact_conv(inputs, n_filters, filter_size=[3, 3], dropout_p=0.2):
    """
    Стандартный предактивационный слой для DenseNets-а, 
    Добавляет пакетную нормализацию, нелинейную ReLU, свёртку и выпадение. 
    """
    
    norm = l.BatchNormalization(inputs=inputs)
    preact = l.relu()(norm)
    conv = l.Conv2D(n_filters, filter_size)(preact)
    if dropout_p != 0.0:
        conv = l.Dropout(1.0-dropout_p)(conv)
    return conv


def DenseBLock(stack, n_layers, growth_rate, dropout_p, scope=None):
    """
    Полносвязный блок для нейросети.
    :param stack входной четырёхмерный тензор
    :param n_layers число внутренних слоёв
    :param growth_rate число карт признаков на внутренний слой
    :return stack четырёхмерный тензор новых карт признаков
    :return new_features четырёхмерный тензор, содержащий ТОЛЬКО новые карты признаков из этого блока
    """
    
    with tf.name_scope(scope) as sc:
        new_features = []
        for i in range(n_layers):
            # Подсчёт новых карт признаков
            layer = preact_conv(stack, growth_rate, dropout_p=dropout_p)
            new_features.append(layer)
            # Укладывание нового слоя
            stack = tf.concat([stack, layer], axis=-1)
        new_features = tf.concat(new_features, axis=-1)
        return stack, new_features
    

def TransitionLayer(inputs, n_filters, dropout_p=0.2, compression=1.0, scope=None):
    """
    Переходный слой
    Добавляет преактивационный блок и 2х2 пуллинг
    """
    
    with tf.name_scope(scope) as sc:
        if compression < 1.0:
            n_filters = tf.to_int32(tf.floor(n_filters*compression))
        l = preact_conv(inputs, n_filters, filter_size=[1, 1], dropout_p=dropour_p)
        l = l.AveragePooling2D((2, 2), strides=(2, 2))(l)
        return l
    

def TransitionDown(inputs, n_filters, dropout_p=0.2, scope=None):
    """
    Слой перехода вниз
    Преактивационный слой + 2х2 максимальный пуллинг
    """
    
    with tf.name_scope(scope) as sc:
        l = preact_conv(inputs, n_filters, filter_size=[1, 1], dropout_p=dropout_p)
        l = l.MaxPooling2D([2, 2])(l)
        return l
    

def TransitionUp(block_to_upsample, skip_connectiion, n_filters_keep, scope=None):
    """
    Слой перехода вверх
    Производит повышающую дискретизацию на block_to_unsample с фактором 2 и складывает с skip_connetion
    """
    
    with tf.name_scope(scope) as sc:
        # ПОвышаюющая дискретизация
        l = l.Conv2DTranspose(n_filters_keep, kernel_size=[3, 3], stride=[2, 2])(block_to_upsample)
        # Сложение с skip_connection
        l = tf.concat([l, skip_connection], axis=-1)
        return l
    

def build_fc_densenet(inputs, preset_model='FC-DenseNet56', num_classes=12, n_filters_first_conv=48, n_pool=5, growth_rate=12, n_layers_per_block=4, dropout_p=.2, scope=None):
    """
    Создание итоговой модели
    :param str preset_model пресетная модель (FC-DenseNet56, FC-DenseNet67, FC-DenseNet103)
    :param int n_classes число классов
    :param int n_filters_first_conv число фильтров для первой свёртки
    :param int n_pool число слоёв пуллинга = слоёв перехода вниз/вверх
    :param int growth_rate число создаваемых новых карт признаков в слое за блок
    :param n_layers_per_block число слоёв на блок. Может быть числом или списком размером 2 * n_pool+1
    :param float dropout_p коэффициент выпадения для каждой свёртки (0 если не нужен)
    """
    
    if preset_model == 'FC-DenseNet56':
        n_pool=5
        growth_rate=12
        n_layers_per_block=4
    elif preset_model == 'FC-DenseNet67':
        n_pool=5
        growth_rate=16
        n_layers_per_block=5
    elif preset_model == 'FC-DenseNet103':
        n_pool=5
        growth_rate=16
        n_layers_per_block=[4, 5, 7, 10, 12, 15, 12, 10, 7, 5, 4]
        
    if type(n_layers_per_block) == list:
        assert (len(n_layers_per_block) == 2 * n_pool + 1)
    elif type(n_layers_per_block) == int:
        n_layers_per_block = [n_layers_per_block] * (2 * n_pool + 1)
    else:
        raise ValueError
        
    with tf.variable_scope(scope, preset_model, [inputs]) as sc:
        
        # Первая свёртка #
        stack = l.Conv2D(inputs, n_filters_first_conv, [3, 3], name='first_conv')
        n_filters = n_filters_first_conv
        
        # Понижаем дискретизацию #
        skip_connection_list = []
        
        for i in range(n_pool):
            # Полносвязный блок
            stack, _ = DenseBlock(stack, n_layers_per_block[i], growth_rate, dropout_p, name='denseblock%d'%(i+1))
            n_filters += growth_rate * n_layers_per_block[i]
            # Под конец полносвязного блока текущий стек сохраняется в skip_connetions_list
            skip_connection_list.append(stack)
            
            # Переход ниже
            stack = TransitionDown(stack, n_filters, dropout_p, name='transitiondown%d'%(i+1))
        
        skip_connection_list = skip_connection_list[::-1]
        
        # Дно нейросети #
        
        # Мы будем поднимать только новые карты признаков
        stack, block_to_upsample = DenseBlock(stack, n_layers_per_block[n_pool], growth_rate, dropout_p, name='denseblockbn%d'%(n_pool+1))
        
        # Повышаем дискретизацию #
        
        for i in range(n_pool):
            # Переход выше (повышение дискр. + сложение с skip_connection)
            n_filters_keep = growth_rate * n_layers_per_block[n_pool + 1]
            stack = TransitionUp(block_to_upsample, skip_connection_list[i], n_filters_keep, name='transitionup%d' % (n_pool + i + 1))
            
            # Мы будем поднимать только новые карты признаков
            stack, block_to_upsample = DenseBlock(stack, n_layers_per_block[n_pool + i + 1], growth_rate, dropout_p, name='denseblock%d' % (n_pool + i + 2))
        
        # Финальный Softmax #
        
        net = l.Conv2D(stack, num_classes, [1, 1], name='logits')
        return net
    